In [1]:
import mysql.connector
import re
import config
import requests
import json
import time


def convert_to_int(string):
    clean_string = re.sub('\W+','', string )
    return int(clean_string)


def rt_rating(movie):
    for rating in movie['Ratings']:
        if rating['Source'] == "Rotten Tomatoes":
            rt_v = rating['Value']
            return convert_to_int(rt_v)
        

def connect_to_db():
    cnx = mysql.connector.connect(
    host=config.host, user=config.user, passwd=config.password, database=config.DB_NAME)
    cursor = cnx.cursor()
    return cnx, cursor


def convert_list_from_tuples(tuple_list):
    """The movie titles are retrieved as a list of tupes, 
    return instead a list with just the title from the first half of each tuple"""
    title_list = []
    for item in tuple_list:
        title_list.append(item[0])
    return title_list
    

def get_all_movie_info():
    select_q = """
                SELECT *
                FROM movies
                """
    cnx, c = connect_to_db()
    c.execute(q)
    results = c.fetchall()
    cnx.close()
    c.close()
    return results

def get_movie_names():
    name_q = """
            SELECT title
            FROM movies"""
    cnx, c = connect_to_db()
    c.execute(name_q)
    results = c.fetchall()
    cnx.close()
    c.close()
    title_list = convert_list_from_tuples(results) 
    return title_list

titles = get_movie_names()
titles[:5]

['Lock, Stock and Two Smoking Barrels',
 'Taxi Driver',
 'Perfect Blue',
 'Star Wars',
 'Wolf Children']

In [2]:
def check_data_fields(data, list_of_fields):
    """Takes in a dictionary and a list of all keys that must be in that dictionary
        returns false if any of those keys are not contained"""
    exists = True
    for field in list_of_fields:
        if field not in data:
            exists = False        
    return exists

def omdb_api(movies):
    """This function retrieves the movie deatils for list of given movies using omdb api"""
    movies_details = []
    for title in movies:
        title.replace(" ", "+")
        url = f'http://www.omdbapi.com/?apikey={config.omdb_api}&t={title}'
        response = requests.get(url)
        movie = response.json()
        #check to make sure each key is always in the dict
        necessary_fields = ['Director', 'BoxOffice', 'Ratings', 'Title']
        if check_data_fields(movie, necessary_fields):
            #4 necessary fields, title is important for the insert statement query later
            movie_dict = {'director': movie['Director'],
                          'boxoffice': movie['BoxOffice'],
                          'rt_rating': rt_rating(movie),
                          'title': movie['Title']}
            movies_details.append(movie_dict)
        time.sleep(.2)
        
    
    return movies_details 
      

In [3]:
two = omdb_api(titles[0:2])
two


[{'director': 'Guy Ritchie',
  'boxoffice': 'N/A',
  'rt_rating': 75,
  'title': 'Lock, Stock and Two Smoking Barrels'},
 {'director': 'Martin Scorsese',
  'boxoffice': 'N/A',
  'rt_rating': 98,
  'title': 'Taxi Driver'}]

In [4]:

def add_omdb_values_to_movies():
    """Add box office, director and rotten tomatoes rating 
    from the omdb to our pre-existing movies DB"""
    
    #retrieve all movie names and pass to omdb_api to get a 
    #list of dictionaries with values of directora nd box_office and rating
    all_titles = get_movie_names()
    omdb_values = omdb_api(all_titles)
    return omdb_values
#     for value_set in omdb_values:
#         insert_omdb_values(list(value_set.values()))

omdb_values = add_omdb_values_to_movies()
omdb_values

[{'director': 'Guy Ritchie',
  'boxoffice': 'N/A',
  'rt_rating': 75,
  'title': 'Lock, Stock and Two Smoking Barrels'},
 {'director': 'Martin Scorsese',
  'boxoffice': 'N/A',
  'rt_rating': 98,
  'title': 'Taxi Driver'},
 {'director': 'Satoshi Kon',
  'boxoffice': 'N/A',
  'rt_rating': 77,
  'title': 'Perfect Blue'},
 {'director': 'George Lucas',
  'boxoffice': 'N/A',
  'rt_rating': 93,
  'title': 'Star Wars: Episode IV - A New Hope'},
 {'director': 'Mamoru Hosoda',
  'boxoffice': 'N/A',
  'rt_rating': 94,
  'title': 'Wolf Children'},
 {'director': 'Kar-Wai Wong',
  'boxoffice': 'N/A',
  'rt_rating': 87,
  'title': 'Chungking Express'},
 {'director': 'Christopher Nolan',
  'boxoffice': '$53,100,000',
  'rt_rating': 76,
  'title': 'The Prestige'},
 {'director': 'Martin Scorsese',
  'boxoffice': '$125,001,000',
  'rt_rating': 68,
  'title': 'Shutter Island'},
 {'director': 'Akira Kurosawa',
  'boxoffice': 'N/A',
  'rt_rating': 96,
  'title': 'Ran'},
 {'director': 'Akira Kurosawa',
  'bo

In [14]:
#to account for when rt_rating or box_office doesn't exist, we have different sql statements for each case
def determing_insert_query(values):
    director = values[0]
    box_office = values[1]
    rt_rating = values[2]
    title = values[3]
    #both box office and rating are absent
    if box_office == 'N/A' and rt_rating == None:
        return f"""UPDATE movies
                    SET 
                        director = "{director}"
                    WHERE title = "{title}";"""
    #box office is absent
    elif box_office == 'N/A':
        return f"""UPDATE movies
                    SET 
                        director = "{director}", 
                        rt_ratings = '{rt_rating}'
                    WHERE title = "{title}";"""
    #rating is absent
    elif rt_rating == None:
        return f"""UPDATE movies
                    SET 
                        director = "{director}", 
                        box_office = '{box_office}'
                    WHERE title = "{title}";"""
    #all accounted for
    else:
        return f"""UPDATE movies
                    SET 
                        director = "{director}", 
                        box_office = '{box_office}', 
                        rt_ratings = '{rt_rating}'
                    WHERE title = "{title}";"""
        

In [15]:
def insert_omdb_values(values):
    insert_q = determing_insert_query(values)
    cnx, c = connect_to_db()
    c.execute(insert_q)
    cnx.commit()
    c.close
    cnx.close

In [16]:

#to test to see if title = {title} actually matches anything
def check_title_match(values):
    title = values[3]
    insert_q = f"""SELECT title, movie_id
            FROM movies
            WHERE title = "{title}";"""
    cnx, c = connect_to_db()
    c.execute(insert_q)
    result = c.fetchall()
    c.close
    cnx.close
    return result
    


In [17]:
for value_set in omdb_values:
    values = list(value_set.values())
    #call check title, and if it's not empty
    if not len(check_title_match(values)) == 0:
        insert_omdb_values(values)

UPDATE movies
                    SET 
                        director = "Guy Ritchie", 
                        rt_ratings = '75'
                    WHERE title = "Lock, Stock and Two Smoking Barrels";
UPDATE movies
                    SET 
                        director = "Martin Scorsese", 
                        rt_ratings = '98'
                    WHERE title = "Taxi Driver";
UPDATE movies
                    SET 
                        director = "Satoshi Kon", 
                        rt_ratings = '77'
                    WHERE title = "Perfect Blue";
UPDATE movies
                    SET 
                        director = "Mamoru Hosoda", 
                        rt_ratings = '94'
                    WHERE title = "Wolf Children";
UPDATE movies
                    SET 
                        director = "Kar-Wai Wong", 
                        rt_ratings = '87'
                    WHERE title = "Chungking Express";
UPDATE movies
                    SET 
                

UPDATE movies
                    SET 
                        director = "Sion Sono", 
                        rt_ratings = '90'
                    WHERE title = "Love Exposure";
UPDATE movies
                    SET 
                        director = "Charles Chaplin", 
                        rt_ratings = '95'
                    WHERE title = "The Circus";
UPDATE movies
                    SET 
                        director = "Luis Buñuel", 
                        rt_ratings = '95'
                    WHERE title = "The Exterminating Angel";
UPDATE movies
                    SET 
                        director = "Anthony Russo, Joe Russo", 
                        box_office = '$664,987,816', 
                        rt_ratings = '85'
                    WHERE title = "Avengers: Infinity War";
UPDATE movies
                    SET 
                        director = "Charles Chaplin", 
                        rt_ratings = '97'
                    WHERE title = "Monsieur Ver

UPDATE movies
                    SET 
                        director = "Quentin Tarantino", 
                        rt_ratings = '92'
                    WHERE title = "Pulp Fiction";
UPDATE movies
                    SET 
                        director = "Joseph L. Mankiewicz", 
                        rt_ratings = '100'
                    WHERE title = "All About Eve";
UPDATE movies
                    SET 
                        director = "Paul Thomas Anderson", 
                        box_office = '$40,133,435', 
                        rt_ratings = '91'
                    WHERE title = "There Will Be Blood";
UPDATE movies
                    SET 
                        director = "Christopher Nolan", 
                        box_office = '$23,844,220', 
                        rt_ratings = '92'
                    WHERE title = "Memento";
UPDATE movies
                    SET 
                        director = "Olivier Nakache, Éric Toledano", 
                       

In [9]:

###TESTING PURPOSES####
#get_5 titles to check their type and formatting
def get_5_titles():
    q = """SELECT title, director
            FROM movies
            LIMIT 5"""
    cnx, c = connect_to_db()
    c.execute(q)
    result = c.fetchall()
    c.close()
    cnx.close()
    return result

get_5_titles()
    

[('Lock, Stock and Two Smoking Barrels', None),
 ('Taxi Driver', None),
 ('Perfect Blue', None),
 ('Star Wars', None),
 ('Wolf Children', None)]

In [11]:
###TESTING PURPOSES####
#get_5 titles to check their type and formatting
def update_random():
    q = """UPDATE movies
            SET 
                director = 'Tom Hardy'
            WHERE title = 'Lock, Stock and Two Smoking Barrels'"""
    cnx, c = connect_to_db()
    c.execute(q)

update_random()

DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction

In [ ]:
get_5_titles